In [1]:
import glob
import os
import site
import sqlite3
import sys
from time import sleep

import logzero
import numpy as np
import pandas as pd
import yaml
from logzero import logger
from tqdm import tqdm
from tqdm.notebook import tqdm
from yaml import dump, load, safe_load

In [2]:
import dash
import dash.dependencies
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import plotly.graph_objects as go
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash

In [3]:
sys.path.append("../../sql")
import queries

sys.path.append("../source")
import dbrd_eda
import dbrd_tools
import plot_tools
import ts_tools

In [4]:
log_path = "logs/"
log_file = "dashboard_eda.log"

logzero.logfile(log_path + log_file, maxBytes=1e6, backupCount=5, disableStderrLogger=True)
logger.info(f"{log_path}, {log_file}\n")

In [5]:
db_path = "../../data/db/"
db_files = dbrd_tools.get_db_files(db_path)
db_files

('_AzCaFlNyTxWaWi.db', '_OkCa.db', 'chino_ca.db', 'okc_ok.db')

In [6]:
configs = None
try:
    with open("../configs/config.yml", "r") as config_in:
        configs = load(config_in, Loader=yaml.SafeLoader)
        logger.info(f"{configs}\n")
except:
    logger.error(f"config file open failure.")
    exit(1)

cfg_vars = configs["url_variables"]
logger.info(f"variables: {cfg_vars}\n")

years = configs["request_years"]
logger.info(f"years: {years}\n")

db_path = configs["file_paths"]["db_path"]

city = configs["location_info"]["city"]
state = configs["location_info"]["state"]
db_file = city + "_" + state + ".db"

db_table1 = configs["table_names"]["db_table1"]
db_table2 = configs["table_names"]["db_table2"]

logger.info(f"{db_path}, {db_file}")

nrows = configs["num_rows"][0]
logger.info(f"number of rows: {nrows}\n")

In [7]:
conn = sqlite3.connect(db_path + db_files[0])
cursor = conn.cursor()

In [8]:
cursor.execute(queries.select_distinct_zips)
distinct_zipcodes = cursor.fetchall()
distinct_zipcodes = [z[0] for z in distinct_zipcodes]
logger.info(f"distinct zip codes:\n{distinct_zipcodes}")
# print(distinct_zipcodes)

In [9]:
zipcode_index = 0
df = ts_tools.get_irr_data(
    conn,
    distinct_zipcodes[zipcode_index],
)
# df.head(5)
df_rsm = df.resample("M").mean()
# df_rsm.head(5)

In [10]:
title1 = "Irradiance Raw Data"
fig1_units = ["Wh", "Wh", "Wh", "Wh", "°C", "%RH", "mm", "m/hr"]
fig1 = plot_tools.plot_data(
    df_rsm,
    title=title1,
    zipcode=distinct_zipcodes[zipcode_index],
    columns=df_rsm.columns.tolist(),
    units=fig1_units,
)

In [11]:
title2 = "Meteorological Conditions"
columns = ["Temperature", "Relative_Humidity", "Precipitable_Water"]
fig2 = plot_tools.plot_multi_line(df_rsm, title=title2, columns=columns)

In [12]:
external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
server = app.server

In [13]:
app.layout = html.Div(
    [
        html.Div([html.H3(), dcc.Graph(id="g1", figure=fig1)], className="six columns"),
        html.Div([html.H3(), dcc.Graph(id="g2", figure=fig2)], className="six columns"),
    ],
    className="row1",
)

In [14]:
if __name__ == "__main__":
    app.run_server(mode="external", host="192.168.64.164", port="8080", use_reloader=False, debug=False)

 * Running on http://192.168.64.164:8080/ (Press CTRL+C to quit)
192.168.64.164 - - [05/Aug/2021 22:59:46] "GET /_alive_2bbd704f-ef95-4227-a3b1-b7154919cc53 HTTP/1.1" 200 -


Dash app running on http://192.168.64.164:8080/


192.168.64.64 - - [05/Aug/2021 22:59:50] "GET / HTTP/1.1" 200 -
192.168.64.64 - - [05/Aug/2021 22:59:51] "GET /_dash-dependencies HTTP/1.1" 200 -
192.168.64.64 - - [05/Aug/2021 22:59:51] "GET /_favicon.ico?v=1.21.0 HTTP/1.1" 200 -
192.168.64.64 - - [05/Aug/2021 22:59:51] "GET /_dash-layout HTTP/1.1" 200 -
192.168.64.64 - - [05/Aug/2021 22:59:51] "GET /_dash-component-suites/dash_core_components/async-graph.js HTTP/1.1" 200 -
192.168.64.64 - - [05/Aug/2021 22:59:51] "GET /_dash-component-suites/dash_core_components/async-plotlyjs.js HTTP/1.1" 200 -
192.168.64.27 - - [05/Aug/2021 23:02:35] "GET / HTTP/1.1" 200 -
192.168.64.27 - - [05/Aug/2021 23:02:35] "GET /_dash-component-suites/dash/deps/prop-types@15.v1_21_0m1627189189.7.2.min.js HTTP/1.1" 200 -
192.168.64.27 - - [05/Aug/2021 23:02:35] "GET /_dash-component-suites/dash/deps/react@16.v1_21_0m1627189189.14.0.min.js HTTP/1.1" 200 -
192.168.64.27 - - [05/Aug/2021 23:02:35] "GET /_dash-component-suites/dash_html_components/dash_html_compo